In [1]:
import numpy as np
import pandas as pd
from numpy import sin, cos, radians, arcsin
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Loading 1 million samples of the data for analysis

In [2]:
data = pd.read_csv("data/train.csv",
                   date_parser=True,
                  nrows=1000000)

## Basic Data Summary

In [3]:
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [4]:
data.fare_amount.describe()

count    1000000.000000
mean          11.348079
std            9.822090
min          -44.900000
25%            6.000000
50%            8.500000
75%           12.500000
max          500.000000
Name: fare_amount, dtype: float64

## date-time conversion and feature creation

In [5]:
data.dtypes  # date parser arg didnt seem to work

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [6]:
data_date_time = data['pickup_datetime'].str[:19]
day_light_saving = - pd.Timedelta(hours=4)
data['EDT'] = pd.to_datetime(data_date_time) - day_light_saving

In [7]:
data['Hour']     = data['EDT'].dt.hour
data['Weekday']  = data['EDT'].dt.day_name()
data['AMorPM']   = np.where(data['Hour']<12,'AM','PM')

In [8]:
data.EDT.min(), data.EDT.max()

(Timestamp('2009-01-01 04:00:46'), Timestamp('2015-07-01 03:53:49'))

In [9]:
data.rename({"key":"MasterDataTime"},axis=1, inplace=True); data.head()

,MasterDataTime,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,EDT,Hour,Weekday,AMorPM
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15 21:26:21,21,Monday,PM
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05 20:52:16,20,Tuesday,PM
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18 04:35:00,4,Thursday,AM
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21 08:30:42,8,Saturday,AM
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09 11:51:00,11,Tuesday,AM


## Try out baseline with the above basic feature engineering

In [10]:
features_of_interest = ["passenger_count", "Hour","Weekday","AMorPM","fare_amount"]
df = data[features_of_interest].copy()

# Numericalising
for col in ["Weekday","AMorPM"]:
    df[col] = df[col].astype("category").cat.codes

feature = df.drop("fare_amount", axis=1).copy()
target = df.fare_amount.copy()

In [11]:
X_train, X_eval, y_train, y_eval = train_test_split(feature,
                                                   target,
                                                   test_size=0.1,
                                                   random_state=0,
                                                   )

In [12]:
X_valid, X_test, y_valid, y_test= train_test_split(X_eval,
                                                  y_eval,
                                                  test_size=0.5,
                                                  random_state=0) 

In [13]:
baseline_LRO = LinearRegression(normalize=True)

In [14]:
model = baseline_LRO.fit(X_train, y_train)

### First Baseline Model Interpretion
Based on our current feature engineering strategy, our model explains about 0.6% of the total variation of the data, which is awful. This means that our model predicts mildly better than merely predicting the mean of the target.

In [15]:
model.score(X_train, y_train)*100

0.06462286984936227

Compare the mean of the target with the RMSE of our prediction. They should match, as the R2 score hinted at.

In [16]:
pred = baseline_LRO.predict(X_valid)

# RMSE
np.sqrt(mean_squared_error(y_valid, pred))

9.66495969297465

In [17]:
# target mean
data.fare_amount.mean()

11.348078720000004

## This indicates that our current features are inadequate at predicting the target.

We now try to make the latitude and longitude more usable

# Creating more features

## Haversine conversion

In [18]:
data = pd.read_csv("data/train.csv",
                   date_parser=True,
                  nrows=1000000)

In [19]:
def kms(df, lat1, long1, lat2, long2):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers
       
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2]-df[lat1])
    delta_lambda = np.radians(df[long2]-df[long1])
     
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c) # in kilometers

    return d

In [20]:
data['kms'] = kms(data, 
                'pickup_latitude',
                'pickup_longitude',
                'dropoff_latitude',
                'dropoff_longitude',
               )

## Extracting features from date including adjusting for DLS

In [21]:
data['EDTdate'] = pd.to_datetime(data['pickup_datetime'].str[:19]) - pd.Timedelta(hours=4)

data['Hour']    = data['EDTdate'].dt.hour
data['AMorPM']  = np.where(data['Hour']<12,'am','pm')
data['Weekday'] = data['EDTdate'].dt.weekday_name
data.head().T

,0,1,2,3,4
key,2009-06-15 17:26:21.0000001,2010-01-05 16:52:16.0000002,2011-08-18 00:35:00.00000049,2012-04-21 04:30:42.0000001,2010-03-09 07:51:00.000000135
fare_amount,4.5,16.9,5.7,7.7,5.3
pickup_datetime,2009-06-15 17:26:21 UTC,2010-01-05 16:52:16 UTC,2011-08-18 00:35:00 UTC,2012-04-21 04:30:42 UTC,2010-03-09 07:51:00 UTC
pickup_longitude,-73.8443,-74.016,-73.9827,-73.9871,-73.9681
pickup_latitude,40.7213,40.7113,40.7613,40.7331,40.768
dropoff_longitude,-73.8416,-73.9793,-73.9912,-73.9916,-73.9567
dropoff_latitude,40.7123,40.782,40.7506,40.7581,40.7838
passenger_count,1,1,2,1,1
kms,1.03076,8.45013,1.38953,2.79927,1.99916
EDTdate,2009-06-15 13:26:21,2010-01-05 12:52:16,2011-08-17 20:35:00,2012-04-21 00:30:42,2010-03-09 03:51:00


## Saving processed file

In [ ]:
data.to_csv("data/processed_train.csv")

## Numericalising

In [ ]:
features_of_interest = ["passenger_count", "kms", "Hour","Weekday","AMorPM","fare_amount"]
df = data[features_of_interest].dropna().copy()

# Numericalising
for col in ["Weekday","AMorPM"]:
    df[col] = df[col].astype("category").cat.codes

feature = df.drop("fare_amount", axis=1).copy()
target = df.fare_amount.copy()

In [ ]:
feature.dtypes

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(feature,
                                                   target,
                                                   test_size=0.1,
                                                   random_state=0,
                                                   )

In [ ]:
X_valid, X_test, y_valid, y_test= train_test_split(X_eval,
                                                  y_eval,
                                                  test_size=0.5,
                                                  random_state=0) 

In [ ]:
baseline_LRO = LinearRegression(normalize=True)
model = baseline_LRO.fit(X_train, y_train)
model.score(X_train, y_train)*100

In [ ]:
pred = baseline_LRO.predict(X_valid)

# RMSE
np.sqrt(mean_squared_error(y_valid, pred))